In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
def load_data(filename):
    df = pd.read_csv(filename)
    return df

In [3]:
# convert timestamp
def convert_timestamp(timestamp):
    timestamp = str(timestamp)
    hour = timestamp[8:10]
    minute = timestamp[10:12]
    second = timestamp[12:14]
    milliseconds = timestamp[14:]
    time = int(hour) * 36000000 + int(minute) * 600000 + int(second) * 10000 + int(milliseconds)
    
    return time

In [4]:
# clean data and get cleaned dataframe
def clean_data(df):
    df['timeIndex'] = df['timestamp'].apply(convert_timestamp)
    base = df['timeIndex'][0]
    df['relativeTime'] = df['timeIndex'] - base
    df['second'] = df['relativeTime'] // 10000
    df_cleaned = df.groupby(['second']).mean()
    del df_cleaned['timestamp']
    del df_cleaned['timeIndex']
    del df_cleaned['relativeTime']
    
    return df_cleaned

In [5]:
# preprocess data
def preprocess_data(filename):
    df = load_data(filename)
    df_cleaned = clean_data(df)
    
    return df_cleaned

In [6]:
# get dataframe for one round play sample
def sample_df(action, emotion, opponent):
    df_action = preprocess_data(action)
    df_emotion = preprocess_data(emotion)
    df_opponent = preprocess_data(opponent)
    df_opponent.columns = ['opponent_up', 'opponent_down', 'opponent_left', 'opponent_right', 'opponent_fire']
    
    size = min(df_action.shape[0], df_emotion.shape[0])
    size = min(size, df_opponent.shape[0])
    
    # pad dataset
    if size <= 100:
        df_action = df_action[df_action.shape[0] - size:]
        df_emotion = df_emotion[df_emotion.shape[0] - size:]
        df_opponent = df_opponent[df_opponent.shape[0] - size:]
    else:
        df_action = df_action[df_action.shape[0] - 100:]
        df_emotion = df_emotion[df_emotion.shape[0] - 100:]
        df_opponent = df_opponent[df_opponent.shape[0] - 100:]
    
    df_emotion.index = df_action.index
    
    df = pd.concat([df_action, df_emotion], axis=1, join_axes=[df_action.index])
    df = pd.concat([df, df_opponent], axis=1, join_axes=[df.index])
    
    return df

In [7]:
# get sample data to be filled in X_train
def fill_Xtrain(action, emotion, opponent, X_train, index):
    df = sample_df(action, emotion, opponent)
    train = df[['up', 'down', 'left', 'right', 'fire', 'joy', 'anger', 'suprise', 
                'opponent_up', 'opponent_down', 'opponent_left', 'opponent_right', 'opponent_fire']].values
#     print(train.shape[0])
    X_train[index, 100-train.shape[0]: , : ] = train

### Get train data

In [31]:
# get X_train
X_train = np.zeros((20, 100, 13))

for i in range(1, 21):
    action = '/Users/cwang/USF/TankGame/action/' + str(i) + '.csv'
    emotion = '/Users/cwang/USF/TankGame/emotion/' + str(i) + '.csv'
    opponent = '/Users/cwang/USF/TankGame/opponent/' + str(i) + '.csv'
    fill_Xtrain(action, emotion, opponent, X_train, i-1)

In [32]:
# get Y_train
Y_train = pd.read_csv('/Users/cwang/USF/TankGame/label.csv', header=None)
Y_train = Y_train[0]

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
Y_train = lb_make.fit_transform(Y_train)

### Get Model

In [33]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [34]:
data_dim = 13
timesteps = 100
#num_classes = 2

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=False,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, Y_train,
          batch_size=2, epochs=1)

Epoch 1/1
20/20 [==============================] - 3s 131ms/step - loss: 5.5798 - acc: 0.6500


### Prediction Example

In [37]:
X_test = np.zeros((1, 100, 13))

for i in range(1, 2):
    action = '/Users/cwang/USF/TankGame/action/' + str(i) + '.csv'
    emotion = '/Users/cwang/USF/TankGame/emotion/' + str(i) + '.csv'
    opponent = '/Users/cwang/USF/TankGame/opponent/' + str(i) + '.csv'
    fill_Xtrain(action, emotion, opponent, X_test, i-1)

In [38]:
X_test[0:1,:,:].shape

(1, 100, 13)

In [39]:
model.predict(X_test[0:1,:,:])

array([[1.]], dtype=float32)